In [3]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout, ReLU
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras import layers, models

def LoadDataWTFPADCW():

    print("Loading defended dataset for closed-world scenario")

    # Point to the directory storing data
    dataset_dir = '/kaggle/input/dataset-defended-close-world/'
    # X represents a sequence of traffic directions
    # y represents a sequence of corresponding label (website's label)
    # Debug: Print dataset directory
    print("Dataset directory:", dataset_dir)

    try:
        # Load training data
        with open(dataset_dir + 'X_train_WTFPAD.pkl', 'rb') as handle:
            X_train = np.array(pickle.load(handle, encoding='latin1'))
        print("X_train loaded")

        with open(dataset_dir + 'y_train_WTFPAD.pkl', 'rb') as handle:
            y_train = np.array(pickle.load(handle, encoding='latin1'))
        print("y_train loaded")

        # Load validation data
        with open(dataset_dir + 'X_valid_WTFPAD.pkl', 'rb') as handle:
            X_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("X_valid loaded")

        with open(dataset_dir + 'y_valid_WTFPAD.pkl', 'rb') as handle:
            y_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("y_valid loaded")

        # Load testing data
        with open(dataset_dir + 'X_test_WTFPAD.pkl', 'rb') as handle:
            X_test = np.array(pickle.load(handle, encoding='latin1'))
        print("X_test loaded")

        with open(dataset_dir + 'y_test_WTFPAD.pkl', 'rb') as handle:
            y_test = np.array(pickle.load(handle, encoding='latin1'))
        print("y_test loaded")

        print("Data dimensions:")
        print("X: Training data's shape : ", X_train.shape)
        print("y: Training data's shape : ", y_train.shape)
        print("X: Validation data's shape : ", X_valid.shape)
        print("y: Validation data's shape : ", y_valid.shape)
        print("X: Testing data's shape : ", X_test.shape)
        print("y: Testing data's shape : ", y_test.shape)

        # Merge datasets
        X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
        y_all = np.concatenate((y_train, y_valid, y_test), axis=0)

        print("Merged data dimensions:")
        print("X: Merged data's shape : ", X_all.shape)
        print("y: Merged data's shape : ", y_all.shape)
        
        # Print features of the merged dataset
        print("Features of the merged dataset:")
        print(X_all)
        
        # Check if the class distribution is balanced
        unique_classes, class_counts = np.unique(y_all, return_counts=True)
        class_distribution = dict(zip(unique_classes, class_counts))

        print("Class distribution:")
        for class_label, count in class_distribution.items():
            print(f"Class {class_label}: {count} samples")

        # Plot the class distribution
       # plt.bar(class_distribution.keys(), class_distribution.values())
       # plt.xlabel('Class Label')
       # plt.ylabel('Number of Samples')
      #  plt.title('Class Distribution')
      #  plt.show()
        
        return X_all, y_all

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Call the function to load and merge data
X_all, y_all = LoadDataWTFPADCW()


Loading defended dataset for closed-world scenario
Dataset directory: /kaggle/input/dataset-defended-close-world/
X_train loaded
y_train loaded
X_valid loaded
y_valid loaded
X_test loaded
y_test loaded
Data dimensions:
X: Training data's shape :  (76000, 5000)
y: Training data's shape :  (76000,)
X: Validation data's shape :  (9500, 5000)
y: Validation data's shape :  (9500,)
X: Testing data's shape :  (9500, 5000)
y: Testing data's shape :  (9500,)
Merged data dimensions:
X: Merged data's shape :  (95000, 5000)
y: Merged data's shape :  (95000,)
Features of the merged dataset:
[[ 1. -1.  1. ... -1. -1.  1.]
 [-1.  1. -1. ...  0.  0.  0.]
 [ 1.  1.  1. ...  0.  0.  0.]
 ...
 [-1.  1. -1. ...  0.  0.  0.]
 [-1.  1. -1. ...  0.  0.  0.]
 [ 1.  1. -1. ...  0.  0.  0.]]
Class distribution:
Class 0: 1000 samples
Class 1: 1000 samples
Class 2: 1000 samples
Class 3: 1000 samples
Class 4: 1000 samples
Class 5: 1000 samples
Class 6: 1000 samples
Class 7: 1000 samples
Class 8: 1000 samples
Class

**adamax + LR = 0.002**

In [4]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.initializers import glorot_uniform
from keras.optimizers import Adamax  # Change here
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import numpy as np

class DFNet:
    @staticmethod
    def build(input_shape, classes, dropout_rate1=0.3, dropout_rate2=0.5, dropout_rate_fc=0.7):
        model = Sequential()

        # ... (previous model definition remains unchanged)
        # Block 1
        filter_num = [None,32, 64, 128, 256]
        kernel_size = [None, 8, 8, 8, 8]
        conv_stride_size = [None, 1, 1, 1, 1]
        pool_stride_size = [None, 4, 4, 4, 4]
        pool_size = [None, 8, 8, 8, 8]

        for i in range(1, 4):
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             input_shape=input_shape if i == 1 else (None, input_shape[1]),
                             name=f'block{i}_conv1'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2 - 1}'))
            model.add(Activation('relu', name=f'block{i}_adv_act1'))
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             name=f'block{i}_conv2'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2}'))
            model.add(Activation('relu', name=f'block{i}_adv_act2'))

            model.add(MaxPooling1D(pool_size=pool_size[i], strides=pool_stride_size[i],
                                   padding='same', name=f'block{i}_pool'))
            model.add(Dropout(0.1, name=f'block{i}_dropout'))

        # ... (rest of the model remains unchanged)

        model.add(Flatten(name='flatten'))

        # Fully connected layers
        for i in range(1, 3):
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name=f'fc{i}'))
            model.add(BatchNormalization(name=f'batch_normalization_{i + 8}'))
            model.add(Activation('relu', name=f'fc{i}_act'))

            model.add(Dropout(dropout_rate_fc, name=f'fc{i}_dropout'))

        # Output layer
        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc_final'))
        model.add(Activation('softmax', name="softmax"))

        print("Model built successfully.")
        return model

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Build the model with the specified dropout rates
model = DFNet.build(input_shape=input_shape, classes=num_classes,
                    dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


# Print the model summary
print("Model Summary:")
model.summary()

# Compile the model
optimizer = Adamax(learning_rate=0.002)  # Change here
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Load and merge data using the function you defined
X_all, y_all =   LoadDataWTFPADCW()
# Define the number of folds
n_splits = 5  # You can adjust this as needed

# Initialize Stratified K-Fold
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results
all_histories = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(stratkf.split(X_all, y_all)):
    print(f"\nTraining on Fold {fold + 1}/{n_splits}")

    # Split the data
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]
    
        # Build the model
    model = DFNet.build(input_shape=input_shape, classes=num_classes,
                        dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
     # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train, y_train, epochs=25, batch_size=64, 
                        validation_data=(X_test, y_test), verbose=2, callbacks=[early_stopping])
    
   


    # Train the model
    #history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nFold {fold + 1} - Test Accuracy: {accuracy * 100:.2f}%")

    # Save history for later analysis if needed
    all_histories.append(history)
    # Check unique labels in y_train
    
    # Update num_classes based on the actual number of unique classes in your dataset
#num_classes = len(np.unique(y_all))
saved_path_keras = '/kaggle/working/DFNet_OpenWorld_WTFPAD.keras'
model.save(saved_path_keras)

print("Trained model saved successfully at:", saved_path_keras)


Model built successfully.
Model Summary:


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv1D)           │ (None, 5000, 32)       │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act1 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv1D)           │ (None, 5000, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act2 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling1D)      │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv1D)           │ (None, 1250, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act1 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv1D)           │ (None, 1250, 64)       │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act2 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling1D)      │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv1D)           │ (None, 313, 128)       │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act1 (Activation)    │ (None, 313, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv1D)           │ (None, 313, 128)       │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act2 (Activation)    │ (None, 313, 128)       │             

 Total params: 5,749,791 (21.93 MB)

 Trainable params: 5,746,847 (21.92 MB)

...

1188/1188 - 92s - 77ms/step - accuracy: 0.6460 - loss: 1.2721 - val_accuracy: 0.7605 - val_loss: 0.8526
Epoch 3/25
1188/1188 - 92s - 77ms/step - accuracy: 0.7183 - loss: 0.9996 - val_accuracy: 0.8019 - val_loss: 0.7005
Epoch 4/25
1188/1188 - 92s - 77ms/step - accuracy: 0.7598 - loss: 0.8503 - val_accuracy: 0.7913 - val_loss: 0.7328
Epoch 5/25
1188/1188 - 92s - 77ms/step - accuracy: 0.7855 - loss: 0.7479 - val_accuracy: 0.8331 - val_loss: 0.5940
Epoch 6/25
1188/1188 - 92s - 77ms/step - accuracy: 0.8074 - loss: 0.6667 - val_accuracy: 0.8348 - val_loss: 0.5910
Epoch 7/25
1188/1188 - 92s - 77ms/step - accuracy: 0.8252 - loss: 0.6010 - val_accuracy: 0.8258 - val_loss: 0.6217
Epoch 8/25
1188/1188 - 92s - 77ms/step - accuracy: 0.8385 - loss: 0.5519 - val_accuracy: 0.8390 - val_loss: 0.5781
Epoch 9/25
1188/1188 - 92s - 77ms/step - accuracy: 0.8511 - loss: 0.5022 - val_accuracy: 0.8705 - val_loss: 0.4706
Epoch 10/25
1188/1188 - 92s - 77ms/step - accuracy: 0.8612 - loss: 0.4638 - val_accuracy: 0

W0000 00:00:1714150272.807610     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


594/594 - 4s - 6ms/step - accuracy: 0.8844 - loss: 0.4487


W0000 00:00:1714150275.968515     106 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Fold 5 - Test Accuracy: 88.44%
Trained model saved successfully at: /kaggle/working/DFNet_OpenWorld_WTFPAD.keras


#using adam

In [ ]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout, ReLU
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras import layers, models
import numpy as np
from keras.layers import Activation

class DFNet:
    @staticmethod
    def build(input_shape, classes, dropout_rate1=0.3, dropout_rate2=0.5, dropout_rate_fc=0.7):
        model = Sequential()

        # ... (previous model definition remains unchanged)
        # Block 1
        filter_num = [None,32, 64, 128, 256]
        kernel_size = [None, 8, 8, 8, 8]
        conv_stride_size = [None, 1, 1, 1, 1]
        pool_stride_size = [None, 4, 4, 4, 4]
        pool_size = [None, 8, 8, 8, 8]

        for i in range(1, 4):
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             input_shape=input_shape if i == 1 else (None, input_shape[1]),
                             name=f'block{i}_conv1'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2 - 1}'))
            model.add(Activation('relu', name=f'block{i}_adv_act1'))
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             name=f'block{i}_conv2'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2}'))
            model.add(Activation('relu', name=f'block{i}_adv_act2'))

            model.add(MaxPooling1D(pool_size=pool_size[i], strides=pool_stride_size[i],
                                   padding='same', name=f'block{i}_pool'))
            model.add(Dropout(0.1, name=f'block{i}_dropout'))

        # ... (rest of the model remains unchanged)

        model.add(Flatten(name='flatten'))

        # Fully connected layers
        for i in range(1, 3):
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name=f'fc{i}'))
            model.add(BatchNormalization(name=f'batch_normalization_{i + 8}'))
            model.add(Activation('relu', name=f'fc{i}_act'))

            # Experiment with different dropout rates
            model.add(Dropout(dropout_rate_fc, name=f'fc{i}_dropout'))

        # Output layer
        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc_final'))
        model.add(Activation('softmax', name="softmax"))

        print("Model built successfully.")
        return model
    # Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Build the model with the specified dropout rates
model = DFNet.build(input_shape=input_shape, classes=num_classes,
                    dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


# Print the model summary
print("Model Summary:")
model.summary()

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Load and merge data using the function you defined
X_all, y_all =   LoadDataWTFPADCW()
# Define the number of folds
n_splits = 5  # You can adjust this as needed

# Initialize Stratified K-Fold
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results
all_histories = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(stratkf.split(X_all, y_all)):
    print(f"\nTraining on Fold {fold + 1}/{n_splits}")

    # Split the data
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]
    
        # Build the model
    model = DFNet.build(input_shape=input_shape, classes=num_classes,
                        dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
     # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train, y_train, epochs=20, batch_size=64, 
                        validation_data=(X_test, y_test), verbose=2, callbacks=[early_stopping])
    
   


    # Train the model
    #history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nFold {fold + 1} - Test Accuracy: {accuracy * 100:.2f}%")

    # Save history for later analysis if needed
    all_histories.append(history)
    # Check unique labels in y_train
    
    # Update num_classes based on the actual number of unique classes in your dataset
#num_classes = len(np.unique(y_all))
saved_path_keras = '/kaggle/working/DFNet_OpenWorld_WTFPAD.keras'
model.save(saved_path_keras)

print("Trained model saved successfully at:", saved_path_keras)

# ****# for Close World Defended Walkie Talkie

In [1]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout, ReLU
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras import layers, models

def LoadDataWALKIETALKIEOW():

    print("Loading defended dataset for closed-world scenario")

    # Point to the directory storing data
    dataset_dir = '/kaggle/input/defended-close-world-walkietalkie-dataset/'
    # X represents a sequence of traffic directions
    # y represents a sequence of corresponding label (website's label)
    # Debug: Print dataset directory
    print("Dataset directory:", dataset_dir)

    try:
        # Load training data
        with open(dataset_dir + 'X_train_WalkieTalkie.pkl', 'rb') as handle:
            X_train = np.array(pickle.load(handle, encoding='latin1'))
        print("X_train loaded")

        with open(dataset_dir + 'y_train_WalkieTalkie.pkl', 'rb') as handle:
            y_train = np.array(pickle.load(handle, encoding='latin1'))
        print("y_train loaded")

        # Load validation data
        with open(dataset_dir + 'X_valid_WalkieTalkie.pkl', 'rb') as handle:
            X_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("X_valid loaded")

        with open(dataset_dir + 'y_valid_WalkieTalkie.pkl', 'rb') as handle:
            y_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("y_valid loaded")

        # Load testing data
        with open(dataset_dir + 'X_test_WalkieTalkie.pkl', 'rb') as handle:
            X_test = np.array(pickle.load(handle, encoding='latin1'))
        print("X_test loaded")

        with open(dataset_dir + 'y_test_WalkieTalkie.pkl', 'rb') as handle:
            y_test = np.array(pickle.load(handle, encoding='latin1'))
        print("y_test loaded")

        print("Data dimensions:")
        print("X: Training data's shape : ", X_train.shape)
        print("y: Training data's shape : ", y_train.shape)
        print("X: Validation data's shape : ", X_valid.shape)
        print("y: Validation data's shape : ", y_valid.shape)
        print("X: Testing data's shape : ", X_test.shape)
        print("y: Testing data's shape : ", y_test.shape)

        # Merge datasets
        X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
        y_all = np.concatenate((y_train, y_valid, y_test), axis=0)

        print("Merged data dimensions:")
        print("X: Merged data's shape : ", X_all.shape)
        print("y: Merged data's shape : ", y_all.shape)
        
        # Print features of the merged dataset
        print("Features of the merged dataset:")
        print(X_all)
        
        # Check if the class distribution is balanced
        unique_classes, class_counts = np.unique(y_all, return_counts=True)
        class_distribution = dict(zip(unique_classes, class_counts))

        print("Class distribution:")
        for class_label, count in class_distribution.items():
            print(f"Class {class_label}: {count} samples")

        # Plot the class distribution
       # plt.bar(class_distribution.keys(), class_distribution.values())
       # plt.xlabel('Class Label')
       # plt.ylabel('Number of Samples')
      #  plt.title('Class Distribution')
      #  plt.show()
        
        return X_all, y_all

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Call the function to load and merge data
X_all, y_all = LoadDataWALKIETALKIEOW()

2024-04-15 04:14:41.759329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 04:14:41.759418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 04:14:41.888997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading defended dataset for closed-world scenario
Dataset directory: /kaggle/input/defended-close-world-walkietalkie-dataset/
X_train loaded
y_train loaded
X_valid loaded
y_valid loaded
X_test loaded
y_test loaded
Data dimensions:
X: Training data's shape :  (80000, 5000)
y: Training data's shape :  (80000,)
X: Validation data's shape :  (5000, 5000)
y: Validation data's shape :  (5000,)
X: Testing data's shape :  (5000, 5000)
y: Testing data's shape :  (5000,)
Merged data dimensions:
X: Merged data's shape :  (90000, 5000)
y: Merged data's shape :  (90000,)
Features of the merged dataset:
[[ 1  1  1 ... -1 -1 -1]
 [ 1 -1  1 ... -1 -1 -1]
 [ 1 -1 -1 ...  0  0  0]
 ...
 [ 1 -1  1 ...  0  0  0]
 [ 1 -1 -1 ...  0  0  0]
 [ 1 -1  1 ...  0  0  0]]
Class distribution:
Class 0: 900 samples
Class 1: 900 samples
Class 2: 900 samples
Class 3: 900 samples
Class 4: 900 samples
Class 5: 900 samples
Class 6: 900 samples
Class 7: 900 samples
Class 8: 900 samples
Class 9: 900 samples
Class 10: 900 sa

**#using Adamax+ LR = 0.002**

In [2]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.initializers import glorot_uniform
from keras.optimizers import Adamax  # Change here
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import numpy as np

class DFNet:
    @staticmethod
    def build(input_shape, classes, dropout_rate1=0.3, dropout_rate2=0.5, dropout_rate_fc=0.7):
        model = Sequential()

        # ... (previous model definition remains unchanged)
        # Block 1
        filter_num = [None,32, 64, 128, 256]
        kernel_size = [None, 8, 8, 8, 8]
        conv_stride_size = [None, 1, 1, 1, 1]
        pool_stride_size = [None, 4, 4, 4, 4]
        pool_size = [None, 8, 8, 8, 8]

        for i in range(1, 4):
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             input_shape=input_shape if i == 1 else (None, input_shape[1]),
                             name=f'block{i}_conv1'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2 - 1}'))
            model.add(Activation('relu', name=f'block{i}_adv_act1'))
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             name=f'block{i}_conv2'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2}'))
            model.add(Activation('relu', name=f'block{i}_adv_act2'))

            model.add(MaxPooling1D(pool_size=pool_size[i], strides=pool_stride_size[i],
                                   padding='same', name=f'block{i}_pool'))
            model.add(Dropout(0.1, name=f'block{i}_dropout'))

        # ... (rest of the model remains unchanged)

        model.add(Flatten(name='flatten'))

        # Fully connected layers
        for i in range(1, 3):
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name=f'fc{i}'))
            model.add(BatchNormalization(name=f'batch_normalization_{i + 8}'))
            model.add(Activation('relu', name=f'fc{i}_act'))

            model.add(Dropout(dropout_rate_fc, name=f'fc{i}_dropout'))

        # Output layer
        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc_final'))
        model.add(Activation('softmax', name="softmax"))

        print("Model built successfully.")
        return model

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Build the model with the specified dropout rates
model = DFNet.build(input_shape=input_shape, classes=num_classes,
                    dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


# Print the model summary
print("Model Summary:")
model.summary()

# Compile the model
optimizer = Adamax(learning_rate=0.002)  # Change here
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Load and merge data using the function you defined
X_all, y_all =   LoadDataWALKIETALKIEOW()
# Define the number of folds
n_splits = 5  # You can adjust this as needed

# Initialize Stratified K-Fold
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results
all_histories = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(stratkf.split(X_all, y_all)):
    print(f"\nTraining on Fold {fold + 1}/{n_splits}")

    # Split the data
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]
    
        # Build the model
    model = DFNet.build(input_shape=input_shape, classes=num_classes,
                        dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


    # Compile the model
    optimizer = Adamax(learning_rate=0.002)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
     # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train, y_train, epochs=25, batch_size=64, 
                        validation_data=(X_test, y_test), verbose=2, callbacks=[early_stopping])
    
   


    # Train the model
    #history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nFold {fold + 1} - Test Accuracy: {accuracy * 100:.2f}%")

    # Save history for later analysis if needed
    all_histories.append(history)
    # Check unique labels in y_train
    
    # Update num_classes based on the actual number of unique classes in your dataset
#num_classes = len(np.unique(y_all))
saved_path_keras = '/kaggle/working/DFNet_OpenWorld_WalkieTalkie.keras'
model.save(saved_path_keras)

print("Trained model saved successfully at:", saved_path_keras)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model built successfully.
Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv1D)           │ (None, 5000, 32)       │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act1 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv1D)           │ (None, 5000, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act2 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling1D)      │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv1D)           │ (None, 1250, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act1 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv1D)           │ (None, 1250, 64)       │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act2 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling1D)      │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv1D)           │ (None, 313, 128)       │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act1 (Activation)    │ (None, 313, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv1D)           │ (None, 313, 128)       │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act2 (Activation)    │ (None, 313, 128)       │             

 Total params: 5,749,791 (21.93 MB)

...

1125/1125 - 102s - 90ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 2/25


W0000 00:00:1713163633.726943     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 3/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 4/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 5/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 6/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 7/25
1125/1125 - 142s - 126ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 8/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 9/25
1125/1125 - 85s - 76ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 10/25
1125/1125 - 142s - 126ms/step - accuracy: 0.0100 - loss: nan - val_accuracy: 0.0100 - val_loss: nan
Epoch 11/25
1125/1125 - 8

W0000 00:00:1713165795.379705     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


563/563 - 3s - 5ms/step - accuracy: 0.0100 - loss: nan

Fold 5 - Test Accuracy: 1.00%
Trained model saved successfully at: /kaggle/working/DFNet_OpenWorld_WalkieTalkie.keras
